# Predicción de comportamientos con Word2vec

## Importando Datos

### Extrayendo la secuencia de Comportamientos Colectivos de los Clientes

In [1]:
import pandas as pd
import gensim
from gensim import corpora, models, similarities

In [2]:
# data = pd.read_csv("../resultados/mobile_consume1_results.csv")
data = pd.read_csv("../../resultados/mobile_consume1_results.csv")

In [3]:
# Lista de Clientes
clientes = data.CO_ID.unique()

In [4]:
# Lista de Periodos para la Opcion 1
year_min = min(data.YEAR) 
year_max = max(data.YEAR) 
week_min = min(data.WEEK) 
week_max = max(data.WEEK) 

periodos = []

if year_min == year_max:
    year = year_min
    for week in range(week_min,week_max+1):
        weeks = ""
        if week < 10:
            weeks = "0"+str(week)
        else:
            weeks = str(week)
        periodos.append(str(year)+"-"+weeks)
        
else:
    for year in range(year_min,year_max+1):
        if year == year_min:
            for week in range(week_min,53):
                
                weeks = ""
                if week < 10:
                    weeks = "0"+str(week)
                else:
                    weeks = str(week)
                periodos.append(str(year)+"-"+weeks)
                #print(year,weeks)
                
        elif year == year_max:   
            for week in range(1,week_max+1):
                
                weeks = ""
                if week < 10:
                    weeks = "0"+str(week)
                else:
                    weeks = str(week)
                periodos.append(str(year)+"-"+weeks)
                #print(year,weeks)
                
        else:    
            for week in range(1,53):
            
                weeks = ""
                if week < 10:
                    weeks = "0"+str(week)
                else:
                    weeks = str(week)
                periodos.append(str(year)+"-"+weeks)
                #print(year,weeks)


## Construyendo CORPUS 

### Opción 1 (considerando "SinActividad")

In [5]:
# Construyendo CORPUS CON CADA TEMPORADA DE CLIENTES (considerando las temporadas vacias)

comportamientos=[]
    
for index, cliente in enumerate(clientes):
    lista = []
    for index, periodo in enumerate(periodos):
        lista.append("SinActividad")
        
    comportamientos.append(lista)
    

In [6]:
lista_periodos = list(periodos)

for i in range(len(data)):
    # en la comportamiendo del cliente i
    fila = data.loc[i,['CO_ID','YEAR','WEEK','COLLECTIVE_CLUSTER']]
    # en el indice del cliente i_c  y del periodo i_p 
    index_c = list(clientes).index(int(fila[0]))
    week = ""
    if int(fila[2]) < 10:
        week = "0" + str(int(fila[2]))
    else:
        week = str(int(fila[2]))
    tem = str(int(fila[1]))+"-"+week
    index_p = lista_periodos.index(tem)
        
    try:
        num = ""
        if int(fila[3]) < 10:
            num = "0" + str(int(fila[3]))
        else:
            num = str(int(fila[3]))
        
        comportamientos[index_c][index_p] = "cluster_"+num
    except Exception as e:
        print("Oops!  That was no valid index.  Try again...")
    

In [7]:
tok_corp = comportamientos

### Opción 2 (sin considerar "SinActividad")

In [8]:
tok_corp2 = []

for cli in clientes:
    data_i = data[(data.CO_ID == cli)]
    lista_i = list(data_i.COLLECTIVE_CLUSTER)

    for j in range(len(lista_i)):
        dat = ""
        if int(str(lista_i[j])) < 10:
            dat = "0" + str(lista_i[j])
        else:
            dat = str(int(fila[2]))
            
        lista_i[j] = "cluster_"+dat
    tok_corp2.append(lista_i)
    

In [9]:
tok_corp2[5]

['cluster_03',
 'cluster_32',
 'cluster_32',
 'cluster_32',
 'cluster_32',
 'cluster_32',
 'cluster_32',
 'cluster_32',
 'cluster_32',
 'cluster_32',
 'cluster_32',
 'cluster_32']

## Modelo

In [33]:
from gensim.models import KeyedVectors

In [11]:
model = gensim.models.Word2Vec(tok_corp, min_count=1, window=5, size=7 , workers=4)

In [12]:
model.vocabulary.max_final_vocab

In [13]:
# Diccionario

# sorted(model.wv.index2word)

In [14]:
# VOCABULARIO

print(sorted(list(model.wv.vocab.keys())))
print("Cantidad = "+str(len(model.wv.vocab)))

['SinActividad', 'cluster_00', 'cluster_01', 'cluster_02', 'cluster_03', 'cluster_04', 'cluster_05', 'cluster_06', 'cluster_07', 'cluster_08', 'cluster_09', 'cluster_10', 'cluster_11', 'cluster_12', 'cluster_13', 'cluster_14', 'cluster_15', 'cluster_16', 'cluster_17', 'cluster_18', 'cluster_19', 'cluster_20', 'cluster_21', 'cluster_22', 'cluster_23', 'cluster_24', 'cluster_25', 'cluster_26', 'cluster_27', 'cluster_28', 'cluster_29', 'cluster_30', 'cluster_31', 'cluster_32', 'cluster_33', 'cluster_34', 'cluster_35', 'cluster_36', 'cluster_37', 'cluster_38', 'cluster_39', 'cluster_40', 'cluster_41', 'cluster_42', 'cluster_43', 'cluster_44', 'cluster_45', 'cluster_46', 'cluster_47', 'cluster_48', 'cluster_49', 'cluster_50', 'cluster_51', 'cluster_52', 'cluster_53', 'cluster_54', 'cluster_55', 'cluster_56', 'cluster_57', 'cluster_58', 'cluster_59', 'cluster_60', 'cluster_61', 'cluster_62', 'cluster_63', 'cluster_64', 'cluster_65', 'cluster_66', 'cluster_67', 'cluster_68', 'cluster_69']
Can

### Resultados

In [15]:
model.predict_output_word(["SinActividad","SinActividad"])

[('cluster_49', 0.021794721),
 ('cluster_25', 0.021263428),
 ('cluster_66', 0.019949611),
 ('cluster_04', 0.019614207),
 ('cluster_15', 0.019488247),
 ('cluster_33', 0.019132052),
 ('cluster_42', 0.019127838),
 ('cluster_27', 0.01908764),
 ('cluster_46', 0.019075947),
 ('cluster_47', 0.019051751)]

In [16]:
model.train([['hello','wordld'],['hello','hello','humf']], total_examples=1, epochs=1)

(0, 5)

In [17]:
model.most_similar("cluster_27")

/home/jkn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/home/jkn/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('cluster_09', 0.9636391997337341),
 ('cluster_41', 0.9567033052444458),
 ('cluster_44', 0.952129602432251),
 ('cluster_35', 0.9502036571502686),
 ('cluster_47', 0.9448263049125671),
 ('cluster_32', 0.9308363199234009),
 ('cluster_59', 0.9084036946296692),
 ('cluster_04', 0.9034002423286438),
 ('cluster_00', 0.8985491991043091),
 ('cluster_19', 0.8889477849006653)]

In [18]:
model.most_similar("cluster_35")

/home/jkn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/home/jkn/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('cluster_32', 0.9930981397628784),
 ('cluster_09', 0.9743544459342957),
 ('cluster_47', 0.9700594544410706),
 ('cluster_41', 0.9656633138656616),
 ('cluster_27', 0.9502036571502686),
 ('cluster_44', 0.9472894668579102),
 ('cluster_69', 0.9378883838653564),
 ('cluster_39', 0.9317026734352112),
 ('cluster_04', 0.9312668442726135),
 ('cluster_59', 0.9311103820800781)]

In [19]:
import random

In [20]:
prueba = []

"""for i in range(3): 
    prueba.append("cluster_"+str(random.randint(0,69))+".0")
    """
prueba = ['cluster_29',"cluster_00", "SinActividad", "SinActividad"]

In [21]:
prueba

['cluster_29', 'cluster_00', 'SinActividad', 'SinActividad']

In [22]:
# Predicion de la siguiente palabra

model.predict_output_word(prueba, topn=5)

[('cluster_27', 0.033819754),
 ('cluster_00', 0.03133183),
 ('cluster_41', 0.030471044),
 ('cluster_04', 0.029714506),
 ('cluster_47', 0.029706858)]

In [23]:
model.predict_output_word(prueba, topn=20)

[('cluster_27', 0.033819754),
 ('cluster_00', 0.03133183),
 ('cluster_41', 0.030471044),
 ('cluster_04', 0.029714506),
 ('cluster_47', 0.029706858),
 ('cluster_44', 0.027837915),
 ('cluster_35', 0.02639033),
 ('cluster_09', 0.024013968),
 ('cluster_69', 0.023763232),
 ('cluster_46', 0.023181709),
 ('cluster_29', 0.022694677),
 ('cluster_34', 0.022310123),
 ('cluster_33', 0.021997245),
 ('cluster_19', 0.021935444),
 ('cluster_32', 0.021859458),
 ('cluster_14', 0.021735394),
 ('cluster_51', 0.021435406),
 ('cluster_37', 0.021334266),
 ('cluster_59', 0.020906426),
 ('cluster_55', 0.020255348)]

In [24]:
# Modelos

model["cluster_00"]

/home/jkn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


array([ 1.349627  ,  1.7760633 ,  2.882443  , -2.6276634 ,  0.66805416,
       -2.716671  , -3.1148107 ], dtype=float32)

In [25]:
model["SinActividad"]

/home/jkn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.20838904,  0.73789966, -0.02116717,  0.18305701,  0.05565107,
        0.11075396, -0.87686175], dtype=float32)

In [26]:
# Parecidos

model.most_similar(["SinActividad"])

/home/jkn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/home/jkn/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('cluster_51', 0.5924944877624512),
 ('cluster_40', 0.5663644671440125),
 ('cluster_59', 0.5621618032455444),
 ('cluster_12', 0.5459011793136597),
 ('cluster_08', 0.5458507537841797),
 ('cluster_07', 0.5406271815299988),
 ('cluster_49', 0.5287001132965088),
 ('cluster_09', 0.5232423543930054),
 ('cluster_57', 0.5221646428108215),
 ('cluster_63', 0.5128747224807739)]

In [27]:
model.syn1neg

/home/jkn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn1neg` (Attribute will be removed in 4.0.0, use self.trainables.syn1neg instead).
  """Entry point for launching an IPython kernel.


array([[-6.15039468e-01, -8.94409299e-01, -2.90683866e-01,
        -3.68683606e-01, -3.51480126e-01, -4.74804729e-01,
         7.13657677e-01],
       [-1.07441711e+00, -1.14973962e+00, -6.51080132e-01,
        -4.89499032e-01, -5.71514785e-01, -2.19496354e-01,
         6.07826054e-01],
       [-6.69174373e-01, -8.86852622e-01, -8.57328549e-02,
        -2.17936471e-01, -1.73064470e-01, -1.81660816e-01,
         1.13148665e+00],
       [-5.67719340e-01, -1.48524153e+00, -2.58248538e-01,
        -2.38022059e-01, -1.63565606e-01, -3.28897029e-01,
         4.68497127e-01],
       [-1.72754169e-01, -1.16660941e+00,  1.12292074e-01,
        -6.87969148e-01, -1.76433213e-02, -8.30398023e-01,
         1.10130274e+00],
       [-6.23133183e-01, -8.97173047e-01,  2.71558791e-01,
        -1.79057911e-01, -4.56858009e-01, -1.01302350e+00,
         1.02903438e+00],
       [-4.02135015e-01, -9.53957081e-01, -3.44399154e-01,
        -7.16827750e-01, -6.80238426e-01, -1.70902801e+00,
         9.2574816

### Referencias

In [ ]:

# https://github.com/RaRe-Technologies/gensim/tree/develop/docs/notebooks
    